# CS352 Machine Learning Coursework

## A: Feature Extraction & Bayes Classifier

### A.1: Extracting, Sampling and Training

A.1.1: ViT Feature Extraction

**Goal of this section:**

In [126]:
import numpy as np
import pandas as pd
import torch
import os
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm.notebook import tqdm

In [127]:
IMAGE_DIR = "celeba_selection"

SAMPLE_SIZE = 20000

ATTRIBUTE_FILE = "list_attr_celeba.txt"

SELECTED_ATTRIBUTES = ["Smiling", "Male", "Young", "Blond_Hair", "Wearing_Hat"]

BATCH_SIZE = 32

PREDICTION_ATTRIBUTE = "Smiling"

RESET = False

RANDOM = True

TRAINING_PERCENT = 0.7

CALIB_PERCENT = 0.15

EMBEDDINGS_FILE = "extracted_features.npz"

RESET = False

In [128]:
sampledImageFiles = os.listdir(IMAGE_DIR)[:SAMPLE_SIZE]

attributeData = pd.read_csv(
    ATTRIBUTE_FILE,
    sep=r'\s+',
    header=1,
    index_col=0
)[SELECTED_ATTRIBUTES]

sampledAttributes = attributeData.loc[sampledImageFiles]

In [129]:
class ImageDataset(Dataset):

    def __init__(self, imageNames, imageDir, transform=None):
        self.imageNames = imageNames
        self.imageDir = imageDir
        self.transform = transform

    def __len__(self):
        return len(self.imageNames)
    
    def __getitem__(self, index):
        if self.transform:
            img = Image.open(os.path.join(self.imageDir, self.imageNames[index])).convert("RGB")
            if self.transform:
                img = self.transform(img)
            return img
        

def extractFeatures(sampledImageFiles, loader, embeddingsFile=EMBEDDINGS_FILE, reset=RESET):

    if not reset and os.path.exists(embeddingsFile):
        data = np.load(embeddingsFile)
        savedImageNames = data["imageNames"]
        if len(savedImageNames) == len(sampledImageFiles) and  np.array_equal(savedImageNames, sampledImageFiles):
            print("Embeddings match sampled image list. Skipping extraction")
            return data["embeddings"]
        


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    vit = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1).to(device)
    vit.eval()
    featureExtractor = create_feature_extractor(vit, return_nodes={"encoder": "features"})

    allFeatures = []

    with torch.no_grad():
    # total = total number of images
        with tqdm(total=len(loader.dataset), desc="Extracting ViT Features", colour="#ebbcba") as pbar:
            for batch in loader:
                batch = batch.to(device)
                features = featureExtractor(batch)["features"][:, 0]
                allFeatures.append(features.cpu())

                # Update progress bar by number of images in the batch
                pbar.update(batch.size(0))

                
    allFeaturesNP = torch.cat(allFeatures, dim=0).numpy()
    print(f"Feature matrix shape: {allFeaturesNP.shape}")

    np.savez(embeddingsFile, embeddings=allFeaturesNP, imageNames=np.array(sampledImageFiles))
    print(f"Embeddings + image names saved to '{embeddingsFile}'.")

    return allFeaturesNP


In [130]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageDataset(sampledImageFiles, IMAGE_DIR, transform=preprocess)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

allFeaturesNP = extractFeatures(sampledImageFiles, loader)

Embeddings match sampled image list. Skipping extraction


In [131]:
from numpy import indices


labels = ((sampledAttributes[PREDICTION_ATTRIBUTE].values + 1) // 2).astype(int)

trainSize = int(TRAINING_PERCENT * SAMPLE_SIZE)
calibSize = int(CALIB_PERCENT * SAMPLE_SIZE)

indices = np.arange(SAMPLE_SIZE)

if RANDOM:
    np.random.shuffle(indices)


trainIndex, calibIndex, testIndex = np.split(indices, [trainSize, trainSize + calibSize])

XTrain, yTrain = allFeaturesNP[trainIndex], labels[trainIndex]
XCalib, yCalib = allFeaturesNP[calibIndex], labels[calibIndex]
XTest, yTest = allFeaturesNP[testIndex], labels[testIndex]

We are going to define a class called GaussianNaiveBayes that will contain the functions for training, and making predictions

In [ ]:
class GaussianNaiveBayes:

    def __init__(self):
        
        self.classes = None
        self.means = None
        self.variances = None
        self.priors = None


    def train(self, X, y):

        # We need to first indentify all our unique values our labels can hold

        self.classes = np.unique(y)
        numFeatures = X.shape[1]

        self.means = np.zeros((len(self.classes), numFeatures))
        self.variances = np.zeros((len(self.classes), numFeatures))
        self.priors = np.zeros(len(self.classes))

        for index, c in enumerate(self.classes):
            Xc = X[y == c]
            self.means[index] = Xc.mean(axis=0)
            self.variances[index] = Xc.var(axis=0) + 1e-6
            self.priors[index] = len(Xc) / len(X)

    
    def __log_guassian_pdf(self, X, mean, variance):
        return -0.5 * (np.log(2 * np.pi * variance) + ((X - mean) ** 2) / variance)

    def predict(self, X):
        probs = self.predictProbabilities(X)
        return self.classes[np.argmax(probs, axis=1)]

        
    def predictProbabilities(self, X):
        logPosteriors = []

        for index in range(len(self.classes)):
            mean = self.means[index]
            variance = self.variances[index]
            logPrior = np.log(self.priors[index])

            # sum across features only
            logLikelihood = np.sum(self.__log_gaussian_pdf(X, mean, variance), axis=1)
            logPosteriors.append(logLikelihood + logPrior)

        logPosteriors = np.vstack(logPosteriors).T
        maxLog = np.max(logPosteriors, axis=1, keepdims=True)
        expScores = np.exp(logPosteriors - maxLog)
        probabilities = expScores / np.sum(expScores, axis=1, keepdims=True)

        return probabilities


Now that the class has been defined we can use it to see how effective the naiive bayes classifier is for this dataset

In [138]:
gnb = GaussianNaiveBayes()

gnb.train(XTrain, yTrain)
yPred = gnb.predict(XTest)
print(XTest[:10][0])
accuracy = np.mean(yPred == yTest)
print(f"Test Accuracy: {accuracy}")

[-8.54981482e-01  1.20940447e+00  1.34680521e+00  1.18113661e+00
  1.01916754e+00 -8.08503568e-01 -3.34579170e-01 -2.47818932e-01
  1.34882939e+00  4.24008965e-01 -8.70176852e-01 -6.90401018e-01
 -4.58931029e-01  7.83703387e-01 -4.21644658e-01  9.94308367e-02
  1.42306641e-01  4.51859206e-01  9.09609944e-02 -1.08289194e+00
  6.21662736e-01  1.53574800e+00 -7.70616114e-01 -3.06403283e-02
 -3.97368789e-01  2.02245370e-01  6.83188558e-01 -1.00186360e+00
 -1.39028740e+00  4.77413803e-01 -7.42653370e-01  6.05231643e-01
  2.07742602e-01  4.26410288e-01 -1.22162616e+00  6.05745196e-01
  8.62082839e-01 -6.73546672e-01  1.76919484e+00 -3.80773157e-01
  6.21171296e-01  6.01671875e-01  9.34109390e-01  4.58661556e-01
 -6.05169535e-02 -3.90735954e-01  4.51705337e-01  6.33170247e-01
 -1.81712234e+00  1.34689736e+00  8.83861482e-01  1.15601927e-01
  1.25418532e+00 -6.24194205e-01 -8.94434929e-01 -1.43443674e-01
  2.32161433e-02 -1.31707454e+00  6.35349929e-01 -3.42251778e-01
  6.07588530e-01 -1.55664

# Part B - Population Calibration

## 1. Compute predicted probabilities and reliability diagram

The first part of this involves modifying the GaussianNaiveBayes class earlier and adding a new function called predictProbabilities which computes the actual probabilities using the softmax function on the liklihood values computed via the predict function. Then we create a new instance of the GNB class using the calibration set and running the function on it.

In [134]:
# We only want the P(smiling) not P(not smiling) so we only use the second value in the sub arrrays


pCalib = gnb.predictProbabilities(XCalib)[:, 1]
print(pCalib[10:20])

[]


A function is now defined that allows us to plot reliability diagram

In [135]:
def plotReliabilityDiagram(yTrue, yProb, numBins=10):
    
    binEdges = numpy.linspace(0, 1, numBins + 1)

    accuracies = []
    confidences = []

    for i in range(numBins):

        index = (yProb >= binEdges[i]) & (yProb < binEdges[i + 1])
        
        if numpy.sum(index) > 0:
            accuracies.append(numpy.mean(yTrue[index]))
            confidences.append(numpy.mean(yProb[index]))
        else:
            accuracies.append(numpy.nan)
            confidences.append(numpy.nan)

    plt.figure(figsize=(6,6))
    plt.plot([0,1],[0,1], 'k--', label="Perfect Calibration")
    plt.plot(confidences, accuracies, marker="o", linestyle="None")
    plt.xlabel("Predicted Probability")
    plt.ylabel("Actual Probability")
    plt.title("Reliability Diagram")
    plt.show()

The function can now be called to display the graph

In [136]:
plotReliabilityDiagram(yCalib, pCalib)

IndexError: boolean index did not match indexed array along axis 0; size of axis is 3000 but size of corresponding boolean axis is 1

## 2. Computing ECE and MSE & Model Assessment

We have to create the 2 functions that compute ECE and MSE.

In [ ]:
def computeECE(yTrue, yProb, numBins=10):

    binEdges = numpy.linspace(0, 1, numBins + 1) 
    binIndices = numpy.digitize(yProb, binEdges) - 1
    ece = 0

    for i in range(numBins):
        
        inBin = (binIndices == i)

        if numpy.sum(inBin) > 0:

            accuracy = numpy.mean(yTrue[inBin])
            confidence = numpy.mean(yProb[inBin])

            ece += (numpy.sum(inBin) / len(yTrue)) * abs(accuracy - confidence)

    return ece



def computeMSE(yTrue, yProb):
    return numpy.mean((yTrue - yProb) ** 2)


As the functions have now been generated we can compute the values based on the test data set.

In [ ]:
pTest = gnb.predictProbabilities(XTest)[:, 1]
ece = computeECE(yTest, pTest)
mse = computeMSE(yTest, pTest)

print(f"ECE: {ece}\nMSE: {mse}")

ECE: 0.26524603262154145
MSE: 0.285733521003401


To determine whether the model is overconfident or underconfident we look at our reliability diagram. If more points are below the diagonal (y = x line) then the model predicts higher probabilities than reality, making it overconfident. The same is the inverse: if mroe points are above the diagonal then the model predicts lower probabilities -> underconfident.

Using the example above with 10 bins, we see that more points are below the line meaning that it is overconfident. This fits as gaussian naive bayes models often underestimates variance and produces sharper, more extreme probabilities near 0 or 1 as a result of the assumptions it makes e.g., conditional independence assumption.

# Part C - Subgroup Calibration

## 1. Plotting reliability, computing ECE, MSE for each subgroup & summary table

First task is to define the subgroups in a format that we can use easily

In [ ]:
# Since the feature set doesnt have features such as female, we assign female as when male is false

SUBGROUPS = {
    "Male": sampledAttributes["Male"] == 1,
    "Female": sampledAttributes["Male"] == 0,
    "Young": sampledAttributes["Young"] == 1,
    "Not Young": sampledAttributes["Young"] == 0,
    "Blond_Hair": sampledAttributes["Blond_Hair"] == 1,
    "Not Blond_Hair": sampledAttributes["Blond_Hair"] == 0,
    "Wearing_Hat": sampledAttributes["Wearing_Hat"] == 1,
    "Not Wearing_Hat": sampledAttributes["Wearing_Hat"] == 0
}